# Задание 1. Написать теггер на данных с русским языком
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы

# Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
1. проверить NER из nltk/spacy/deeppavlov
2. написать свой нер попробовать разные подходы
   1. передаём в сетку токен и его соседей
   2. передаём в сетку только токен
3. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

In [40]:
# imports
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
from nltk.corpus import stopwords
import pymorphy2
import re

from tqdm.notebook import tqdm

from corus import load_ne5

import matplotlib
%matplotlib inline

## Написать теггер на данных с русским языком

Работать с русским языком будем на скачанных данных из Роспотребнадзора

In [4]:
df = pd.read_csv('../Lesson_04/question.csv')

In [5]:
df.head(2)

,Unnamed: 0,question
0,0,Получила не качественную услугу в «Центр косме...
1,1,Ведётся продажа контрафакта г.к Анапа ул Астра...


In [36]:
words_regex = re.compile('\w+')

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

morph = pymorphy2.MorphAnalyzer()

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')


def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stureiko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /Users/stureiko/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


In [14]:
preprocessed_text = list(tqdm(map(preprocess, df['question']), total=len(df)))

  0%|          | 0/245 [00:00<?, ?it/s]

In [17]:
words = list(tqdm(map(find_words, df['question']), total=len(df)))

  0%|          | 0/245 [00:00<?, ?it/s]

In [45]:
train_data = words[1:int(len(words) * 0.9)]
test_data = words[int(len(words) * 0.9):]
test_sent = words[0]

In [46]:
train_data = [nltk.pos_tag(i, lang = 'rus') for i in train_data]
test_data = [nltk.pos_tag(i, lang = 'rus') for i in test_data]
# test_sent = nltk.pos_tag(test_sent, lang = 'rus')

In [47]:
unigram_tagger = UnigramTagger(train_data)
# display(unigram_tagger.tag(test_sent))
unigram_tagger.evaluate(test_data)

/var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/ipykernel_27030/1149866346.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))


[('получила', 'V'),
 ('качественную', None),
 ('услугу', 'S'),
 ('центр', 'S'),
 ('косметологии', None),
 ('лица', 'S'),
 ('или', 'CONJ'),
 ('ооо', 'S'),
 ('тиэльбьюти', None),
 ('киров', 'S'),
 ('красноармейская', None),
 ('процедура', 'S'),
 ('увеличение', None),
 ('губ', None),
 ('филлером', None),
 ('выполнял', None),
 ('услугу', 'S'),
 ('директор', 'S'),
 ('салона', None),
 ('лысова', None),
 ('татьяна', 'S'),
 ('евгеньевна', None),
 ('путем', 'S'),
 ('долгих', None),
 ('переговоров', None),
 ('через', 'PR'),
 ('письменную', None),
 ('претензию', 'S'),
 ('денежные', 'A=pl'),
 ('средства', 'S'),
 ('вернули', 'V'),
 ('очень', 'ADV'),
 ('хотели', None),
 ('возвращать', 'V'),
 ('утверждая', None),
 ('что', 'CONJ'),
 ('это', 'S-PRO'),
 ('моя', 'A-PRO=f'),
 ('вина', None),
 ('хотя', 'CONJ'),
 ('этого', 'S-PRO'),
 ('делала', None),
 ('данную', 'A-PRO=f'),
 ('процедуру', None),
 ('других', 'A-PRO=pl'),
 ('косметологов', None),
 ('всё', 'S-PRO'),
 ('было', 'V'),
 ('хорошо', 'ADV'),
 ('дост

0.6776171703898379

In [56]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
# display(bigram_tagger.tag(test_sent))
bigram_tagger.evaluate(test_data)

/var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/ipykernel_27030/523054775.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  bigram_tagger.evaluate(test_data)


0.6811213315812528

In [57]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
# display(trigram_tagger.tag(test_sent))
trigram_tagger.evaluate(test_data)

/var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/ipykernel_27030/3676292912.py:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  trigram_tagger.evaluate(test_data)


0.6811213315812528

In [52]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data) 

/var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/ipykernel_27030/2738799135.py:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tag.evaluate(test_data)


0.6811213315812528

___В отличии от английского языка   для русского nltk работает хуже и на триграме и комбинации тегеров не дает прироста качества___

## Написать свой теггер как на занятии

Сделаем словарик русских имен

In [53]:
dir = 'Collection5/'
records = load_ne5(dir)


t_df = pd.DataFrame(columns = ['index', 'type', 'text'])
for item in records:
    for i in item.spans:
        if i.type == 'PER':
            t_df.loc[len(t_df)] = [i.index, i.type, i.text] 

names = list(set(t_df['text']))

Помимо полных имен (ФИО) выделим только имена и добавим их в список

In [59]:
full_n = []
for s in tqdm(names):
    full_n.append(s)
    full_n.append(nltk.word_tokenize(s)[0])

  0%|          | 0/1423334 [00:00<?, ?it/s]

In [60]:
from nltk.tag import SequentialBackoffTagger

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in full_n])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'NNP'
        else:
             return None


In [62]:
nt = NamesTagger()
print(nt.tag(['Екатерина'])) 
print(nt.tag(['Александр Рогачев'])) 

[('Екатерина', 'NNP')]
[('Александр Рогачев', 'NNP')]


# NER

## Natasha

In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [7]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = df['question'][0]
doc = Doc(text)

In [8]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[0]
sent.morph.print()

            Получила VERB|Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
                  не PART|Polarity=Neg
        качественную ADJ|Case=Acc|Degree=Pos|Gender=Fem|Number=Sing
              услугу NOUN|Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
                   в ADP
                   « PUNCT
               Центр NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
        косметологии NOUN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
                лица NOUN|Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
                   » PUNCT
                 или CCONJ
                 ООО PROPN|Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
                   « PUNCT
          ТИЭЛЬБЬЮТИ NOUN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
                   » PUNCT
                   г NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   . PUNCT
               Киров PROPN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
     Красноармейская ADJ|C

In [9]:
sent.syntax.print()

             Получила        
          ┌► не              advmod
        ┌►└─ качественную    amod
      ┌─└─── услугу          
      │ ┌──► в               case
      │ │ ┌► «               punct
  ┌─┌─└►└─└─ Центр           nmod
  │ │ │ └──► косметологии    nmod
  │ │ └────► лица            nmod
  │ └──────► »               punct
  │       ┌► или             cc
┌─└──►┌─┌─└─ ООО             conj
│     │ │ ┌► «               punct
│     │ └►└─ ТИЭЛЬБЬЮТИ      appos
│     │ └──► »               punct
│     └──►┌─ г               appos
│     ┌──►│  .               punct
│     │ ┌─└► Киров           amod
└────►│ │    Красноармейская appos
      │ └──► 41              appos
      │      .               
      └─┌─── Процедура       
        │ ┌► »               punct
      ┌─└►└─ Увеличение      appos
      │ └►┌─ губ             nmod
      │   └► филлером        nmod
      └────► »               punct
             .               
